In [1]:
from models import *
from utilities import Utilities as helper
from utilities import AverageMeter
from data_loader import trainloader
from data_loader import testloader




=> Building model...
Files already downloaded and verified
Files already downloaded and verified


In [2]:
model_name = "resnet20_quan"
resnet20_model_quan = resnet20_quant().cuda()

In [ ]:
lr = 7e-3
weight_decay = 1e-4
epochs = 100
# pre_best_prec = 85.470

criterion = nn.CrossEntropyLoss().cuda()

res20_optimizer = torch.optim.SGD(resnet20_model_quan.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
pre_best_prec = helper.train_model(resnet20_model_quan, model_name=model_name, optimizer=res20_optimizer, trainloader=trainloader, testloader=testloader, criterion=criterion, epochs=epochs, pre_best_prec=pre_best_prec)


In [8]:
helper.test_model(model=resnet20_model_quan, model_name=model_name, testloader=testloader)

Test: [0/79]	Time 0.190 (0.190)	Loss 0.2105 (0.2105)	Prec 93.750% (93.750%)
 * Prec 90.380% 


In [4]:
w_bit = 4
quan_con_layer = next(resnet20_model_quan.layer1[0].modules()).conv1
weight_q = quan_con_layer.weight_q # quantized value is stored during the training
w_alpha = quan_con_layer.weight_quant.wgt_alpha.data.item()  # alpha is defined in your model already. bring it out here
w_delta = w_alpha/(2**(w_bit-1)-1)   # delta can be calculated by using alpha and w_bit
weight_int = weight_q/w_delta # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers


# conv.weight  = torch.nn.Parameter(torch.randn(n_ch_out,n_ch_in,k_size,k_size))
# ##########################


# weight_quant = weight_quantize_fn(w_bit= w_bits)  ## define quant function
# weight_quant.wgt_alpha = torch.tensor(w_alpha)
# w_quant      = weight_quant(conv.weight)
# w_delta      = w_alpha/(2**(w_bits-1)-1)
# w_int        = w_quant/w_delta


tensor([[[[-0.0000,  0.0000,  0.0000],
          [ 0.0000,  1.0000,  1.0000],
          [-1.0000, -0.0000,  1.0000]],

         [[ 2.0000, -0.0000, -1.0000],
          [-1.0000, -4.0000, -5.0000],
          [ 1.0000, -3.0000, -5.0000]],

         [[ 2.0000,  2.0000,  2.0000],
          [ 3.0000,  3.0000,  2.0000],
          [ 2.0000,  3.0000,  2.0000]],

         ...,

         [[-0.0000, -4.0000, -1.0000],
          [-2.0000, -5.0000, -0.0000],
          [-1.0000, -3.0000,  1.0000]],

         [[ 1.0000, -3.0000, -3.0000],
          [ 2.0000, -3.0000, -4.0000],
          [ 3.0000, -1.0000, -2.0000]],

         [[ 3.0000,  2.0000,  1.0000],
          [ 3.0000,  3.0000,  4.0000],
          [ 0.0000,  1.0000,  2.0000]]],


        [[[-5.0000, -7.0000, -5.0000],
          [ 1.0000,  2.0000,  0.0000],
          [ 2.0000,  6.0000,  4.0000]],

         [[ 1.0000,  1.0000,  2.0000],
          [-0.0000, -4.0000, -2.0000],
          [ 1.0000, -1.0000, -3.0000]],

         [[ 1.0000,  2.0000, -0

In [5]:
from saveOutput import SaveOutput

resnet_save_out = SaveOutput()
blocks_res = SaveOutput.hook(resnet20_model_quan.layer1.modules(), QuantConv2d, resnet_save_out)

prehooked
prehooked
prehooked
prehooked
prehooked
prehooked


In [6]:
def act_quantization(b):

    def uniform_quant(x, b=3):
        xdiv = x.mul(2 ** b - 1)
        xhard = xdiv.round().div(2 ** b - 1)
        return xhard

    class uq(torch.autograd.Function):   # here single underscore means this class is for internal use

        def forward(ctx, input, alpha):
            input_d = input/alpha
            input_c = input_d.clamp(max=1)  # Mingu edited for Alexnet
            input_q = uniform_quant(input_c, b)
            ctx.save_for_backward(input, input_q)
            input_q_out = input_q.mul(alpha)
            return input_q_out

    return uq().apply


In [9]:
x = resnet_save_out.outputs[0][0]# input of the 2nd conv layer
x_bit = 4    

x_alpha  = 3.284
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 


tensor([[[[ 2.0000,  4.0000,  4.0000,  ...,  5.0000,  5.0000,  3.0000],
          [ 2.0000,  4.0000,  4.0000,  ...,  4.0000,  4.0000,  3.0000],
          [ 3.0000,  4.0000,  4.0000,  ...,  3.0000,  4.0000,  3.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 4.0000,  1.0000,  1.0000,  ...,  0.0000,  0.0000,  1.0000],
          [10.0000,  7.0000,  8.0000,  ...,  7.0000,  6.0000,  4.0000],
          [ 9.0000,  7.0000,  7.0000,  ...,  8.0000,  6.0000,  4.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  2.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  

In [10]:
conv_quan = nn.Conv2d(16, 16, kernel_size=(3, 3), padding=(1, 1), bias=False)
conv_quan.weight = torch.nn.Parameter(weight_int)

psum_int = conv_quan(x_int)
print("psum int:", psum_int)
psum_recovered = psum_int*w_delta*x_delta

psum int: tensor([[[[ 3.6000e+01,  1.5300e+02,  1.7800e+02,  ...,  1.7400e+02,
            1.1400e+02,  5.1000e+01],
          [-8.8000e+01,  1.6000e+01,  3.4000e+01,  ...,  1.5000e+01,
           -9.0000e+00, -1.6000e+01],
          [-1.1100e+02,  7.0000e+00, -5.0000e+00,  ...,  2.0000e+00,
            1.3000e+01,  2.0000e+01],
          ...,
          [ 2.4800e+02,  2.1200e+02,  2.1300e+02,  ...,  1.6900e+02,
            1.3900e+02, -3.3000e+01],
          [ 1.7400e+02,  1.8600e+02,  1.8900e+02,  ...,  1.5000e+02,
            1.2800e+02, -3.8000e+01],
          [ 1.0600e+02,  1.2200e+02,  1.2800e+02,  ...,  1.4100e+02,
            9.7000e+01,  2.6000e+01]],

         [[ 1.3500e+02,  2.2400e+02,  2.6300e+02,  ...,  2.4900e+02,
            2.8600e+02,  2.7000e+01],
          [-3.5000e+01,  2.5000e+01,  4.4000e+01,  ...,  9.0000e+00,
            1.3000e+01, -1.0600e+02],
          [ 2.7000e+01,  6.5000e+01,  6.5000e+01,  ...,  1.0000e+01,
            3.5000e+01, -7.3000e+01],
          

In [11]:
conv_unquan = nn.Conv2d(16, 16, kernel_size=(3, 3), padding=(1, 1), bias=False)
conv_unquan.weight = torch.nn.Parameter(weight_q)
out = conv_unquan(x)


In [12]:
diff = abs(out - psum_recovered)
print("gap", diff)

gap tensor([[[[5.1659e-01, 6.7050e-01, 9.4139e-01,  ..., 1.8837e-01,
           5.8331e-01, 2.4019e-01],
          [1.1429e-01, 2.0802e-01, 3.8295e-01,  ..., 6.7232e-01,
           2.7058e-01, 7.9369e-01],
          [1.9863e-01, 3.1256e-02, 2.1388e-01,  ..., 1.1955e+00,
           4.6978e-01, 5.2290e-01],
          ...,
          [3.4516e-01, 6.7371e-01, 8.7989e-01,  ..., 6.0625e-01,
           2.7849e-01, 5.2161e-01],
          [4.2660e-01, 4.7936e-01, 1.2529e-01,  ..., 2.2255e-01,
           4.5503e-01, 2.8423e+00],
          [1.8825e-02, 4.2435e-01, 2.9205e-01,  ..., 7.0882e-02,
           8.7108e-01, 9.2278e-01]],

         [[4.3692e-01, 2.9545e-01, 1.6597e-01,  ..., 6.1315e-01,
           2.3375e-01, 4.9136e-01],
          [9.9156e-01, 8.3958e-02, 4.3507e-01,  ..., 3.2550e-01,
           3.4743e-01, 7.3152e-01],
          [1.1248e+00, 2.6149e-02, 6.1545e-01,  ..., 1.2539e+00,
           1.1430e+00, 4.0929e-01],
          ...,
          [4.0891e-01, 1.4092e-01, 3.0194e-01,  ..., 1.

In [13]:
diff.mean()

tensor(0.7316, device='cuda:0', grad_fn=<MeanBackward0>)

In [16]:
resnet20_2bit_quan = resnet20_quant(w_quan_bits=2).cuda()
pre_best_prec = 0
quan_model_name = "2bit_quan_resnet"

In [ ]:
lr = 7e-2
weight_decay = 1e-4
epochs = 100
# pre_best_prec = 85.470

criterion = nn.CrossEntropyLoss().cuda()

res20_2bit_optimizer = torch.optim.SGD(resnet20_2bit_quan.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
pre_best_prec = helper.train_model(resnet20_2bit_quan, model_name=quan_model_name, optimizer=res20_2bit_optimizer, trainloader=trainloader, testloader=testloader, criterion=criterion, epochs=epochs, pre_best_prec=pre_best_prec)


In [17]:
helper.test_model(model=resnet20_2bit_quan, model_name=quan_model_name, testloader=testloader)

Test: [0/79]	Time 0.233 (0.233)	Loss 0.3729 (0.3729)	Prec 87.500% (87.500%)
 * Prec 83.960% 
